In [1]:
import sys
import os
path = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path)
sys.path.append(path + '/api')

import geopandas as gpd
from api.app.jhm_metric_calcs.jhm_metric import main
import networkx as nx

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# read dataset with avg rent price and house locations (geometry)
city_name = 'saint-petersburg'
g_type='drive'

# read drive nx graph
G_d = nx.read_graphml(f'/home/gk/vscode/rita_project_4/industrial-location/api/app/provisions_data/{city_name}_prov/G_{g_type}.graphml')

house_prices = gpd.read_parquet(f"/home/gk/vscode/rita_project_4/industrial-location/api/app/provisions_data/{city_name}_prov/houses_price_demo_prov.parquet")
house_prices = house_prices.to_crs(G_d.graph["crs"])



In [3]:
G_d.graph['crs']

'epsg:32636'

In [4]:
list(G_d.nodes(data=True))[3]

('1', {'x': 351066.09, 'y': 6643191.31})

In [5]:
# define company's potential location
# and convert coordinates crs to local crs

company_location = {"lon": 59.869047, "lat": 30.206686}

In [6]:
class Worker():
    speciality: str
    salary: int

worker1 = Worker()
worker1.speciality = "worker_1"
worker1.salary = 10_000

worker2 = Worker()
worker2.speciality = "worker_2"
worker2.salary = 80_000

In [7]:
worker_and_salary = [worker1, worker2]

In [8]:
res = main(
    gdf_houses=house_prices,
    worker_and_salary=worker_and_salary,
    company_location=company_location,
    graph=G_d,
    cell_size_meters=300)



 K: 0.516 

 D: 0.516 

all K4 coef [0, 1.032]


In [18]:
import pandas as pd
import json

json.loads(json.dumps(res["K4"]))

{'worker_1_avg': 0, 'worker_2_avg': 1.032}

In [10]:
import json

gpd.GeoDataFrame.from_features(
    json.loads((res["gdfs"])["worker_2"]["grid"]), crs=G_d.graph["crs"]
).explore(column="Idx_mean", cmap="RdYlGn", zoom=14, tiles="cartodbpositron")